In [35]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Preparing the training set and the test set
training_set = np.array(pd.read_csv('ml-100k/u1.base', delimiter='\t'), dtype=int)
test_set = np.array(pd.read_csv('ml-100k/u1.test', delimiter='\t'), dtype=int)

# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0]))) # must check both traing and test set for max number
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))# must check both traing and test set for max number


# create the matrix of users = lines/rows and movies = col/features
def convert(data):
    new_data = []
    for id_users in range(1,nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users ] # get all the movies by the user_id
        id_rating = data[:,2][data[:,0] == id_users ] # get all the rating by the user_id
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_rating
        new_data.append(list(ratings))
    return new_data

# convert and change to Torch Tensors
training_set = torch.FloatTensor(convert(training_set))
test_set = torch.FloatTensor(convert(test_set))

In [39]:
# create the architecture of the neural network
class SAE(nn.Module): # inheritance
    def __init__(self, ):
        super(SAE, self).__init__() # use this to access the inherited functions
        self.fc1 = nn.Linear(nb_movies, 20) # first full connection: input features, nodes in layer
        self.fc2 = nn.Linear(20, 10) # second layer input is eqaul to first layer nodes
        self.fc3 = nn.Linear(10, 20) # third layer start to decode
        self.fc4 = nn.Linear(20, nb_movies) # output layer same number of outputs as original inputs
        self.activation = nn.Sigmoid() # activation function
    def forward(self, x): # encoding
        x = self.activation(self.fc1(x)) 
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x) # don't need to use activation function on output layer
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr= 0.01, weight_decay=0.5)


In [42]:
# Training the SAE optimized

nb_epoch = 200

for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
       # create batch of single vector online learning, update weights on every input vector
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        # optimization technique only look at users that have rated the movie
        if torch.sum(target.data > 0) > 0:
            output = sae(input) # make prediction
            target.require_grad = False # don't compute gradient, optimization technique
            output[target == 0] = 0 # values won't count in the computation of error :. optimize by not using them
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) # stop denominator from being 0
            loss.backward() # decided to increase of decrease
            train_loss += np.sqrt(loss.item() * mean_corrector)
            s += 1.
            optimizer.step() # decides the intensity of the increase of decrease
    print('epoch: ', str(epoch), " loss: ", str(train_loss/s))


epoch:  1  loss:  1.7598484071728857
epoch:  2  loss:  1.0962776981164422
epoch:  3  loss:  1.0535493613959406
epoch:  4  loss:  1.038243894273267
epoch:  5  loss:  1.0309641064925403
epoch:  6  loss:  1.0265935410882168
epoch:  7  loss:  1.0237039728830963
epoch:  8  loss:  1.0221266222421563
epoch:  9  loss:  1.0207291565560141
epoch:  10  loss:  1.0197919737482286
epoch:  11  loss:  1.0189784572233325
epoch:  12  loss:  1.0184690323302705
epoch:  13  loss:  1.0177094236231943
epoch:  14  loss:  1.0176259127786826
epoch:  15  loss:  1.0173172179765642
epoch:  16  loss:  1.0169021940279352
epoch:  17  loss:  1.0168245627895258
epoch:  18  loss:  1.0164545832905074
epoch:  19  loss:  1.016537269331242
epoch:  20  loss:  1.016091363820186
epoch:  21  loss:  1.0160634366032077
epoch:  22  loss:  1.0158503001373378
epoch:  23  loss:  1.0157894622157873
epoch:  24  loss:  1.0158709087163682
epoch:  25  loss:  1.0155187620737038
epoch:  26  loss:  1.0156708999178314
epoch:  27  loss:  1.015

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
     # create batch of single vector online learning, update weights on every input vector
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    # optimization technique only look at users that have rated the movie
    if torch.sum(target.data > 0) > 0:
        output = sae(input) # make prediction
        target.require_grad = False # don't compute gradient, optimization technique
        output[target == 0] = 0 # values won't count in the computation of error :. optimize by not using them
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) # stop denominator from being 0
        test_loss += np.sqrt(loss.item() * mean_corrector)
        s += 1.
print("test loss: ", str(test_loss/s))












